## 🔄 Ajuste do Diretório de Trabalho

Antes de carregar ou manipular arquivos, é importante garantir que estamos no diretório correto do projeto.  
O código abaixo verifica se o notebook está sendo executado a partir da pasta `notebooks`. Se for o caso, ele sobe um nível na hierarquia de diretórios para garantir que o diretório de trabalho seja a raiz do projeto.

Isso é útil para manter caminhos relativos consistentes ao acessar dados, scripts ou outros recursos do projeto.

📌 **Resumo do que o código faz:**
- Verifica se o diretório atual termina com `notebooks`.
- Se sim, volta uma pasta (para a raiz do projeto).
- Exibe o novo diretório de trabalho.


In [1]:
import os

# Verifica se o diretório de trabalho atual termina com 'notebooks'
if os.path.basename(os.getcwd()) == 'notebooks':
    # Se sim, sobe um nível de diretório para a pasta raiz do projeto
    os.chdir('..')

# Imprime o diretório de trabalho para confirmar que a mudança foi feita
print(f"Diretório de Trabalho Atual: {os.getcwd()}")

Diretório de Trabalho Atual: c:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov


## 📦 Carregamento das Bibliotecas para a Fase 3: Modelagem Preditiva

Nesta etapa, carregamos todas as bibliotecas necessárias para realizar o treinamento, validação e interpretação de modelos de Machine Learning aplicados à previsão da tendência do Ibovespa.

---

### 🔧 Principais Componentes Importados:

#### 📊 Manipulação de Dados
- `pandas`, `numpy`: Estruturação e transformação de dados tabulares e numéricos.
- `duckdb`: Consulta e carregamento eficiente da base persistida na fase anterior.

#### ⚙️ Modelagem e Avaliação
- `lightgbm`: Framework de gradient boosting eficiente, usado para modelagem supervisionada.
- `sklearn.model_selection.train_test_split`: Divisão da base de forma temporal para simular previsão realista.
- `sklearn.metrics`: Avaliação com métricas como `accuracy`, `ROC AUC`, `confusion_matrix`.

#### 🧠 Interpretação do Modelo
- `shap`: Framework de interpretabilidade para entender a importância das features no modelo treinado.

#### 📈 Visualização
- `matplotlib`, `seaborn`: Criação de gráficos e análise visual dos resultados.

#### 🛠️ Configuração do Projeto
- `src.config`: Importa o caminho e demais parâmetros definidos nas fases anteriores.

---

✅ Todas as bibliotecas e dependências da **Fase 3 - Modelagem** foram carregadas com sucesso e estão prontas para uso.


In [2]:
import duckdb
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
from sklearn.model_selection import train_test_split # Usaremos para a divisão temporal
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance

# Importa nossas configurações de projeto
import src.config as config

# Helpers do Notebook
from IPython.display import display

# Configurações de estilo
sns.set_theme(style='whitegrid', palette='viridis')
plt.style.use("fivethirtyeight")
%matplotlib inline

print("✅ Bibliotecas para a Fase 3 carregadas com sucesso!")

c:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Bibliotecas para a Fase 3 carregadas com sucesso!
